# Ionic and Acid Naming Quiz

In [ ]:
from IPython.display import display, Latex, Math, HTML, Markdown
import pandas as pd
from numpy import *
from numpy.random import *
import ipywidgets as widgets
from ChemicalFormulaState_W22 import Formula

In [ ]:
# load ion list from csv file into 'data'

data = pd.read_csv('ion list.csv', usecols = ['name', 'formula'])

$$\require{mhchem}$$ 


### Instructions

Fill in the missing formula or name in the table below by typing your answer into the answer boxes.  When you type the <ENTER> key, your answer will be formatted and graded.  Only ionic names and acid names are allowed -- binary molecular names are not yet included.  (*e.g.* $\ce{H2S}$ must be named "hydrosulfuric acid" and not "hydrogen sulfide")
    
#### Names
    
Use only lowercase letters.  Be careful with spaces: remember, no spaces in cation names (*e.g.*, $\ce{Co^{2+}}$ is "cobalt(II)", not "cobalt (II)")
    
#### Formulas
    
Type in symbols, numbers, and/or parentheses as needed; your answer will get formatted (if it is entered correctly) when you hit the <ENTER> key.  For example, to enter $\ce{Fe(NO_3)_2}$, you would literally type "Fe(NO3)2".

In [ ]:
cations = data[['+' in item for item in data['formula'].values]]
anions = data[['-' in item for item in data['formula'].values]]

# list of anions to avoid when cation is H+
banned_anions = ["hydride", "hydroxide", "oxide", "peroxide", "nitride", "arsenide", "phosphide"]
allowed = anions
for name in banned_anions:
    allowed = allowed.drop(allowed[allowed.name == name].index)
    

In [ ]:
def name_acid_from_anionName(anion_name):
    stem, suffix = anion_name[:-3], anion_name[-3:]
    
    # stem modifications
    if stem == "sulf": stem = "sulfur"
    if stem == "phosph": stem = "phosphor"

    name_dict = {'ate':f"{stem}ic acid",
                 'ite':f"{stem}ous acid",
                 'ide':f"hydro{stem}ic acid"}
   
    name = name_dict.get(suffix, "name unknown")
    return name

In [ ]:
# uses pandas lookup to translate a string ion name into the string formula from 
# the Excel file containing all of the ions
def get_formula(ion_name):
    try:
        idx = pd.Index(data.name).get_loc(ion_name)
        return Formula(data.formula[idx])
    except KeyError:
        print(f"{ion_name} not in database")

In [ ]:
def create_formula_from_ions(cation, anion):
    '''Assume that cation and anion are Formula objects'''
    N = lcm(cation.charge, abs(anion.charge))
    cat_mult = N // cation.charge
    str_cat_mult = "_" + str(cat_mult) if cat_mult > 1 else ""
    an_mult = N // abs(anion.charge)
    str_an_mult = "_" + str(an_mult) if an_mult > 1 else ""
    cat_n_atoms = sum(list(cation.elements.values()))
    an_n_atoms = sum(list(anion.elements.values()))
    
    #create new Formula
    str_cat = cation.formula.split('^')[0]
    if cat_n_atoms > 1 and cat_mult > 1:
        str_cat = "(" + str_cat + ")"
    str_an = anion.formula.split('^')[0]
    if an_n_atoms > 1 and an_mult > 1:
        str_an = "(" + str_an + ")"
    str_compound = f"{str_cat}{str_cat_mult}{str_an}{str_an_mult}"
    return Formula(str_compound)
    

In [ ]:
def _create_acid_formula(strAcid):
    # stem adjustment
    def stemmod(stem):
        if stem == "phosphor": stem = "phosph"
        if stem == "sulfur": stem = "sulf"
        return stem
    
    if strAcid.endswith("ic"):
        if strAcid.startswith("hydro"):
            stem = stemmod(strAcid[5:-2])
            name = f"{stem}ide"
        else:
            stem = stemmod(strAcid[:-2])
            name = f"{stem}ate"  
    elif strAcid.endswith("ous"):
        stem = stemmod(strAcid[:-3])
        name = f"{stem}ite"
    else:
        print("acid unknown")
        return None
    anion = get_formula(name)
    if anion:
        return create_formula_from_ions(Formula("H^+"), anion)
    else:
        print("acid unknown")

In [ ]:
# put together parts to create a formula from one written name
def create_formula_from_name(strName):
    try:
        strCation, strAnion = strName.split()
        if strAnion == "acid":
            return _create_acid_formula(strCation)
        
        if strCation == "hydrogen":
            print("name as acid, not ionic!")
            return None
        cation = get_formula(strCation)
        if not cation:
            print(f"cation name '{strCation}' not recognized")
            return None
        anion = get_formula(strAnion.strip())
        if not anion:
            print(f"anion name '{strAnion}' not recognized")
            return None
        return create_formula_from_ions(cation, anion)
    except ValueError:
        print("name must consist of two words; only one space allowed")
        if '(' in strName:
            idx = strName.index('(')
            if strName[idx-1] == ' ':
                newName = strName[:idx-1] + strName[idx:]
                # print("pieces: ", '"%s"' % strName[:idx-1]," & ", '"%s"' % strName[idx:])
                print(f"remove space from cation name; using '{newName}' instead")
                return create_formula_from_name(newName)
        else:
            print("name not recognized")

In [ ]:
def check_answer(proposed_answer, answer, answer_type = 'name'):
    if answer_type == 'name':
        f_test = create_formula_from_name(proposed_answer)
        output = f_test.formula == answer if f_test else False
    elif answer_type == 'formula':
        output = proposed_answer == answer.replace("_", "")
    else:
        print("Error in check_answer")
    return output

def update(change):
    idx = proposed_answers.index(change.owner)
    form = forms[answer_types[idx]]
    outputs[idx].value = form.format(change.new)
    if check_answer(change.new, answers[idx], answer_types[idx]):
        response = "<b><font color='green'>Correct!</b>"
    else:
        response = "<b><font color='red'>wrong</b>"
    response_widgets[idx].value = response

out = widgets.Output(layout={'border': '1px solid black'})

# reset containers

proposed_answers = []
answers = []
outputs = []
response_widgets = []
answer_types = []

def reset_quiz(N=10):
    global answer_types, proposed_answers, answers, outputs, response_widgets
    # clear output
    out.clear_output()
    
    # select indices for N cations and N anions
    ctns = choice(cations.index, N, replace=True)
    ans = choice(anions.index, N, replace=True)
    
    # correct for any H+ cations in list:
    for idx in where(ctns == 0):
        ans[idx] = choice(allowed.index)
    
    # reset containers
    proposed_answers = []
    answers = []
    outputs = []
    response_widgets = []
    
    # make random selection of whether to show name or formula
    answer_choice = ['formula', 'name']
    answer_types = choice(answer_choice, N, replace=True)
    
    box_layout = widgets.Layout(display='flex',
                               flex_flow = 'row',
                               align_items='stretch',
                               width='70%')
    
    # format strings
    form = r"$$\ce{{ {} }}$$"
    forms = {'formula':r"$$\ce{{ {} }}$$",
            'name':r"<font size=3>{}"}


    grid = widgets.GridspecLayout(N+1,5)
    grid[0,0] = widgets.Label("Name")
    grid[0,1] = widgets.Label("Formula")
    grid[0,2] = widgets.Label("Your answer", layout = widgets.Layout(display='flex', justify_content='center'))

    for i in range(N):
        c_i = ctns[i]
        a_i = ans[i]
        catname = cations.name[c_i]
        cat = cations.formula[c_i]
        anname = anions.name[a_i]
        an = anions.formula[a_i]
        Name = name_acid_from_anionName(anname) if catname == "hydrogen" else catname + " " + anname
        f = create_formula_from_ions(Formula(cat), Formula(an))

        # insert display objects here
        if answer_types[i] == 'formula':
            item1 = widgets.HTML(value = f"<font size=3>{Name}")
            item2 = widgets.HTMLMath(value =rf"")
            item35 = widgets.HTMLMath(value = "")
            answers.append(f.formula)
        else:
            item1 = widgets.HTML(value = "")
            item2 = widgets.HTMLMath(value =rf"$ \ce{{ {f.formula} }}$")
            item35 = widgets.HTML("")
            answers.append(f.formula)
        item3 = widgets.Text(value = "", description="Answer:", placeholder="Type answer <ENTER>", continuous_update=False) 
        proposed_answers.append(item3)
        outputs.append(item35)
        item3.observe(update, names='value')
        item4 = widgets.HTML(value = "") 
        response_widgets.append(item4)
        items = [item1, item2, item3, item35, item4]
        for j, item in enumerate(items):
            grid[i+1,j] = item

    with out:
        display(grid)

def on_button_clicked(b):
    N = intQuestions.value
    reset_quiz(N)
    with out:
        display(hbox)
    
btnReset = widgets.Button(description="Reset Quiz", button_style="success")
btnReset.on_click(on_button_clicked)
intQuestions = widgets.BoundedIntText(value = 10, description = "# questions", min = 5, max = 20, step=1)
hbox = widgets.HBox([btnReset, intQuestions])

on_button_clicked("test")
out

In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>

